In [ ]:
import _base_path
import pickle
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from resources.data_io import load_mappings
from resources.metrics import ConfusionMatrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
DATA                = 'incidents'
MODELS              = [
    'bow-rnd',
    'bow-sup',
    'bow-knn',
    'bow-lr',
    'bow-svm',
    'tfidf-knn',
    'tfidf-lr',
    'tfidf-svm',
    'roberta-base',
    'xlm-roberta-base'
]
METRICS             = {
    'micro-f1':     lambda y_true, y_pred: f1_score(y_true, y_pred, average='micro', zero_division=0.0),
    'macro-f1':     lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro', zero_division=0.0),
    'precision':    lambda y_true, y_pred: precision_score(y_true, y_pred, average='macro', zero_division=0.0),
    'recall':       lambda y_true, y_pred: recall_score(y_true, y_pred, average='macro', zero_division=0.0),
#    'accuracy':     lambda y_true, y_pred: accuracy_score(y_true, y_pred)
}
LABEL               = 'hazard-category'
CV_SPLITS           = [0, 1, 2, 3, 4]

# Load Class-Mappings:

In [ ]:
class_map = load_mappings(f"../data/{DATA}/splits/", LABEL)
class_map

In [ ]:
counts = pd.read_csv(f'../data/{DATA}/{DATA}_final.csv')[LABEL].value_counts()

class_map = list(zip(
    class_map,
    range(len(class_map)),
    [counts[c] if c in counts else 0 for c in class_map]
))
class_map.sort(key=lambda row:row[2], reverse=True)
class_map

In [ ]:
classes_all = [c for c, _, n in class_map if n > 0]
classes_all

In [ ]:
with open(f'../data/{DATA}/support_zones.json', 'r') as file:
    classes_high_support, classes_low_support = json.load(file)[LABEL]

In [ ]:
classes_high_support

In [ ]:
classes_low_support

## Filter classes:

In [ ]:
for split in CV_SPLITS:
    with open(f"../data/{DATA}/splits/split_{LABEL.split('-')[0]}_{split:d}.pickle", "rb") as f:
        # load data for split:
        data = pickle.load(f)

        # get unique classes in train and test sets:
        c_train = [c for c, i, _ in class_map if sum(data['train'][LABEL].values == i) >= 4]
        c_test  = [c for c, i, _ in class_map if sum(data['test'][LABEL].values == i) >= 1]

    # only use classes that are present in the train AND test set:
    classes_all          = [c for c in classes_all if c in c_train and c in c_test]
    classes_high_support = [c for c in classes_high_support if c in c_train and c in c_test]
    classes_low_support  = [c for c in classes_low_support if c in c_train and c in c_test]

len(classes_all)

# Load Results:

In [ ]:
results = {}

for m in MODELS:
    r = []
    try:
        for split in CV_SPLITS:
            with open(f'{m}/{m}-{LABEL}-{split:d}.pickle', 'rb') as f:
                r.append(pickle.load(f))
    except FileNotFoundError: continue
    results[m] = r

In [ ]:
def calculate_metrics(classes):
    classes = [i for c, i, _ in class_map if c in classes]
    metrics = {}

    for model in results:
        metrics[model] = {metric: np.empty(len(CV_SPLITS), dtype=float) for metric in METRICS}

        for split, r in enumerate(results[model]):
            mask = np.vectorize(lambda c: c in classes)(r['labels'])
            y_true = np.stack([r['labels'][mask] == c for c in classes], dtype=int, axis=1)
            y_pred = np.stack([r['predictions'][mask] == c for c in classes], dtype=int, axis=1)

            for metric in METRICS:
                metrics[model][metric][split] = METRICS[metric](y_true, y_pred)

    return metrics

In [ ]:
metrics_all = calculate_metrics(classes_all)
metrics_high_support = calculate_metrics(classes_high_support)
metrics_low_support = calculate_metrics(classes_low_support)

In [ ]:
def metric2latex(metrics_dict, report_max=True): 
    metrics = np.array([[metrics_dict[model][metric] for metric in metrics_dict[model]] for model in metrics_dict], dtype=float)
    
    avg     = metrics.mean(axis=-1)
    best    = np.round(avg, 2) == np.round(np.max(avg, axis=0), 2)
    if metrics.shape[-1] == 1: return np.vectorize(
        lambda a, b:    f'\\cellcolor\u007Bblue!15\u007D\\footnotesize $\\bf {a:.2f}$'
                        if b else  f'\\footnotesize ${a:.2f}$'
    )(avg, best)

    if report_max:
        return np.vectorize(
            lambda a, m, b: f'\\cellcolor\u007Bblue!15\u007D\\footnotesize $\\bf {a:.2f}$ & \\cellcolor\u007Bblue!15\u007D\\footnotesize $\\bf {m:.2f}$'
                            if b else f'\\footnotesize ${a:.2f}$ & \\footnotesize ${m:.2f}$'
        )(avg, metrics.max(axis=-1), best)

    else:
        err     = np.abs(metrics - avg.reshape(avg.shape + (1,))).mean(axis=-1)
        return np.vectorize(
            lambda a, e, b: f'\\cellcolor\u007Bblue!15\u007D\\footnotesize $\\bf {a:.2f}$ \\tiny $\\bf\\pm {e:.2f}$'
                            if b else f'\\footnotesize ${a:.2f}$ \\tiny $\\pm {e:.2f}$'
        )(avg, err, best)

In [ ]:
ltx_all = metric2latex(metrics_all)
ltx_hs  = metric2latex(metrics_high_support)
ltx_ls  = metric2latex(metrics_low_support)

for i, model in enumerate(MODELS):
    row =  f'{model.upper()} &\n'

    if model in metrics_all:            row += ' & '.join(ltx_all[i])
    else:                               row += ' &'*(len(METRICS)-1)
    row += ' &\n'

    if model in metrics_high_support:   row += ' & '.join(ltx_hs[i])
    else:                               row += ' &'*(len(METRICS)-1)
    row += ' &\n'

    if model in metrics_low_support:    row += ' & '.join(ltx_ls[i])
    else:                               row += ' &'*(len(METRICS)-1)
    row += ' \\\\\n'

    print(row)

# Plot confusion matrix

In [ ]:
MODEL = 'tfidf-lr'

hs_mask = np.zeros(len(class_map), dtype=bool)
ls_mask = np.zeros(len(class_map), dtype=bool)
ms_mask = np.zeros(len(class_map), dtype=bool)

for c, i, _ in class_map:
    hs_mask[i] = c in classes_high_support
    ls_mask[i] = c in classes_low_support
    ms_mask[i] = not(hs_mask[i] or ls_mask[i])
        
y_pred = np.array([
    np.argmax([hs_mask[e], ms_mask[e], ls_mask[e]])
    for e in results[MODEL][0]['predictions']
])
y_true = np.array([
    np.argmax([hs_mask[e], ms_mask[e], ls_mask[e]])
    for e in results[MODEL][0]['labels']
])

cm = ConfusionMatrix(y_true, y_pred, classes=["High", "Medium", "Low"])

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(3, 3))
cm.plot(axs)
fig.savefig(f'../pictures/plots/cm_{MODEL}_{LABEL}.pdf')